In [ ]:
import os
from glob import glob
from collections import defaultdict
import MDAnalysis as mda
from MDAnalysis.analysis import distances
import pandas as pd
import statistics
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

def get_user_input(prompt):
    """
    Get user input from the command line.
    """
    while True:
        user_input = input(prompt)
        if user_input.lower() == 'q':
            return None
        try:
            return user_input
        except ValueError:
            print("Invalid input. Please enter a number or 'q' to quit.")

def select_residues(structure, a1, a2):
    """
    Selects atoms from a specific residue range with given atom names.
    """
    return [
        structure.select_atoms(f"resid {i} and (name CG or name CZ or name CD or name CE or name O or name N)")
        for i in range(int(a1), int(a2)+1)
    ]


def calculate_distances(res_peptide, res_protein):
    """
    Calculates distance array for all residue pairs in time series.
    """
    return [
        
        distances.distance_array(peptide.atoms.positions, protein.atoms.positions)
        for peptide, protein in zip(res_peptide, res_protein)
    ]


def analyze_distances(a1_peptide, a1_protein, res_peptide, res_protein, distances, threshold=4.0):
    """
    Analyzes distances for each frame and calculates frequency of close contacts.
    """
    mix = []
    mix = [(i, j) for i in range(len(res_peptide)) for j in range(len(res_protein))]
    

    dic = defaultdict(list)
    for i in range(len(distances)):
        for j in range(len(distances[i])):
            for k in range(len(distances[i][j])):
                for l in range(len(distances[i][j][k])):
                    key = (
                        f"generic_distance_{int(mix[i][0]) + int(a1_peptide)}_{int(mix[i][1]) + int(a1_protein)}_"
                        f"{res_peptide[mix[i][0]].atoms.names[k]}_{res_protein[mix[i][1]].atoms.names[l]}")
                    dic[key].append(distances[i][j][k][l])
    freq = []
    for key, values in dic.items():
        if any(val <= threshold for val in values):
            pd.DataFrame(dic[key]).to_csv(key + ".csv", index=False)
            df = pd.read_csv(key + ".csv", header=None, skiprows=1)
            count_below_threshold = (df <= threshold).sum(axis=0)
            freq.append(count_below_threshold / len(df))
    return freq


def count_files_in_subdirs(subdirs):
    # Create an empty list to store the file paths
    file_paths = []
    list_files = []
    common_distance = []

    # Loop through the subdirectories and find the file paths and count the number of occurrences of file in the subdirectories
    for subdir in subdirs:
        new_subdir = subdir + "/generic_distance"
        for files in os.listdir(new_subdir):
            if files.startswith("generic_distance_"):
                list_files.append(files)
    line_counter = Counter(list_files)

    # Print the number of occurrences of each line and extract residue IDs (assuming 3rd and 4th elements)
    new_dic = defaultdict(set)
    for line, count in line_counter.items():
        print(f'{count}: {line.split("_")[2:4]}')
        if count > 1:
            residue_ids = line.split("_")[2:4]
            for res_id in residue_ids:
                residue_name = structure.select_atoms("resid {}".format(res_id)).resnames[0]
                new_dic[residue_name].add(res_id)
            # save in common_distance if the line is not already in common_distance
                if line not in common_distance:
                    common_distance.append(line)
        else:
            print(f'No common distances found for {line.split("_")[2:4]}')
    return common_distance , new_dic


def get_names(residue_data, new_dic):
    """
    Extracts residue names from residue data.
    """
    for key, value in new_dic.items():

        if residue_data[2] in value:
            f = key + str(list(value)[0])
        if residue_data[3] in value:
            s = key + str(list(value)[0])
    return f, s


def gen_freq(filepath):
    """
    Reads distance data from a file, calculates frequencies.
    """ 
    below_threshold = (df[1:] <= 4.5).sum(axis=0)
    return below_threshold / len(df) 


def plot_distances(subdirs, common_distance):
    """
    Plot distances over time.
    """
    
    colors = ['pink', 'green', 'blue', 'orange', 'slateblue']
    num_rows = len(common_distance)

    fig, axs = plt.subplots(num_rows, 2, figsize=(17, num_rows * 4),
                            gridspec_kw={'width_ratios': [1, 5], 'hspace': 0.4, 'wspace': 0.1})
    mpl.rcParams["legend.loc"] = 'upper right' 

    for i, line in enumerate(common_distance): 
        
        # Get the residue names for the title
        original_name = line.split("_")
        new_name = get_names(original_name)
        title = f"{new_name[0]}, {original_name[4]} - {new_name[1]}, {original_name[5].split('.')[0]}"
        axs[i, 1].set_title(title)
        axs[i, 0].set_title("Distances")
        
        j = 0
        single_freq = []
        dfs = []

        # Loop through the subdirectories and plot the distances
        for subdir in subdirs:
            subdir_path = os.path.join(subdir, 'generic_distance')
            
            for filename in os.listdir(subdir_path):
                if filename == line: 
                    filepath = os.path.join(subdir_path, filename)
                    file = pd.read_csv(filepath, header=None, skiprows=1)
                    single_freq.append(gen_freq(file))
                    dfs.append(file)
                    df = pd.read_csv(filepath, skiprows=1, header=None).values.ravel() # Read the distance data
                    
                    n_frames = len(df) # Get the number of frames
                    axs[i, 1].set_xlim(0, n_frames) 
                    axs[i, 1].axhline(y = 4.0, color='r', linestyle='-') # 4 angstrom threshold

                    # Set the y-axis labels
                    axs[i,1].set_ylabel('Distance (nm)')
                    max = df.max()
                    min = df.min()
                    distance = np.arange(int(min.min()), int(max.max()+3),1) # Assuming distance data in single file
                    nm = distance * 0.1 # convert to nm
                    axs[i,1].set_yticks(distance)
                    axs[i,1].set_yticklabels(f'%.1f' % nm for nm in nm)
                    
                    # Set the x-axis labels
                    axs[i,1].set_xlabel('Time (ns)')
                    time = np.arange(0, n_frames,100)  # Assuming 100 frames per ns  
                    ns = time * 0.3 # convert to ns
                    axs[i,1].set_xticks(time)
                    axs[i,1].set_xticklabels(f'%d' % ns for ns in ns)
                    
                    # Plot the frequency of distances below 4.0 nm
                    axs[i, 0].set_ylim(0, 1)
                    axs[i, 0].set_ylabel('Frequency')
                    axs[i, 0].set_xticks([])
                    axs[i, 0].scatter(1, single_freq[j], color= colors[subdirs.index(subdir)-1]) 
                    
                    # Plot the distance data
                    axs[i, 1].plot(df, label=subdir, color=colors[subdirs.index(subdir) - 1])
                    
                    # Increment the index
                    j += 1
            
                handles, labels = axs[i, 1].get_legend_handles_labels()
                if labels:
                    axs[i, 1].legend()
                
            # write the mean frequency of the distances
            freq = gen_freq(pd.concat(dfs)) 
            axs[i,0].bar(1, freq, width = 0.9, color = 'b', alpha = 0.2)
    plt.show()


def main():
    """
    Main function to analyze distances between peptide and protein residues.
    """
    adding_subdirs = True
    subdirs = []
    while adding_subdirs:
        input_dir = get_user_input("Insert subdirectory name (or 'q' to quit): ")
        if input_dir is None:
            break
        elif os.path.exists(input_dir):
            subdirs.append(input_dir)
        else:
            print("The subdirectory does not exist.")

    print("Subdirectories:", subdirs)

    for directory in subdirs:
        pdb_files = glob(os.path.join(directory, "*.pdb"))
        if not pdb_files:
            print(f"No PDB files found in {directory}")
            continue

        structure = mda.Universe(pdb_files[0])

        a1_protein = get_user_input("Insert the first residue number of the protein: ")
        if a1_protein is None:
            continue
        a2_protein = get_user_input("Insert the last residue number of the protein: ")
        if a2_protein is None:
            continue
        a1_peptide = get_user_input("Insert the first residue number of the peptide: ")
        if a1_peptide is None:
            continue
        a2_peptide = get_user_input("Insert the last residue number of the peptide: ")
        if a2_peptide is None:
            continue

        res_protein = select_residues(structure, a1_protein, a2_protein)
        res_peptide = select_residues(structure, a1_peptide, a2_peptide)

        distances = []
        for ts in structure.trajectory:
            distances.append(calculate_distances(res_peptide, res_protein))

        print(f"Analysis results for {directory}")
    
        freq = analyze_distances(a1_peptide, a1_protein,res_peptide, res_protein, distances)
    
    common_distance = count_files_in_subdirs(subdirs)
    
    plot_distances(subdirs, common_distance)
    

if __name__ == "__main__":
    main()
